# Stereo Reconstruction

## Preparations

In [2]:
import pgmpy
import cv2
import skimage
import os
import time
import numpy

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
# load the images from dataset-2005 subdirectories
def load_image():
    # load images
    left_images = []
    right_images = []
    for folder in os.listdir('dataset-2005'):
        if os.path.isdir('dataset-2005/' + folder):
            for subfolder in os.listdir('dataset-2005/' + folder):
                for file in os.listdir('dataset-2005/'+folder+'/'+subfolder):
                    if file.startswith('view1'):
                        left_img = cv2.imread('dataset-2005/'+folder+'/'+subfolder+'/'+file)
                        left_images.append(left_img)
                    elif file.startswith('view5'):
                        right_img = cv2.imread('dataset-2005/'+folder+'/'+subfolder+'/'+file)
                        right_images.append(right_img)
    return left_images, right_images

In [44]:
left_images, right_images = load_image()

In [1]:
def view_images(left_images, right_images):
    for i in range(len(left_images)):
        cv2.imshow('left', left_images[i])
        cv2.imshow('right', right_images[i])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [45]:
view_images(left_images, right_images)